In [208]:
# 使用SimpleTagBased算法对Delicious2K数据进行推荐
# 原始数据集：https://grouplens.org/datasets/hetrec-2011/
# 数据格式：userID     bookmarkID     tagID     timestamp
import random
import math
import operator
import pandas as pd


# 数据加载
# 字典类型，保存了user对item的tag，即{userid: {item1:[tag1, tag2], ...}}
def load_data():
    records={}
    file_path = "./user_taggedbookmarks-timestamps.dat"
    print("开始数据加载...")
    df = pd.read_csv(file_path, sep='\t')
    for i in range(len(df)):
        uid = df['userID'][i]
        iid = df['bookmarkID'][i]
        tag = df['tagID'][i]
        # 键不存在时，设置默认值{}
        records.setdefault(uid,{})
        records[uid].setdefault(iid,[])
        
        records[uid][iid].append(tag)
    print("数据集大小为 %d." % (len(df)))
    print("设置tag的人数 %d." % (len(records)))
    print("数据加载完成\n")
    return records
records=load_data()

开始数据加载...
数据集大小为 437593.
设置tag的人数 1867.
数据加载完成



In [209]:
# 将数据集拆分为训练集和测试集
def train_test_split(records,ratio, seed=100):
    random.seed(seed)
    # 训练集，测试集
    train_data = {}
    test_data = {}
    for u in records.keys():
        for i in records[u].keys():
            # ratio比例设置为测试集
            if random.random()<ratio:
                test_data.setdefault(u,{})
                test_data[u].setdefault(i,[])
                for t in records[u][i]:
                    test_data[u][i].append(t)
            else:
                train_data.setdefault(u,{})
                train_data[u].setdefault(i,[])
                for t in records[u][i]:
                    train_data[u][i].append(t)        
    print("训练集样本数 %d, 测试集样本数 %d" % (len(train_data),len(test_data)))
    return test_data,train_data

In [210]:
test_data,train_data=train_test_split(records,0.2, seed=100)

训练集样本数 1860, 测试集样本数 1793


In [211]:
# 设置矩阵 mat[index, item] = 1
def addValueToMat(mat, index, item, value=1):
    if index not in mat:
        mat.setdefault(index,{})
        mat[index].setdefault(item,value)
    else:
        if item not in mat[index]:
            mat[index][item] = value
        else:
            mat[index][item] += value


# 使用训练集，初始化user_tags, tag_items, user_items
def initStat(records,train_data):
    user_tags = {}
    tag_items = {}
    user_items = {}
    records=train_data
    for u,items in records.items():
        for i,tags in items.items():
            for tag in tags:
                #print tag
                # 用户和tag的关系
                addValueToMat(user_tags, u, tag, 1)
                # tag和item的关系
                addValueToMat(tag_items, tag, i, 1)
                # 用户和item的关系
                addValueToMat(user_items, u, i, 1)
    print("user_tags, tag_items, user_items初始化完成.")
    print("user_tags大小 %d, tag_items大小 %d, user_items大小 %d" % (len(user_tags), len(tag_items), len(user_items)))
    return user_tags,tag_items,user_items

In [212]:
user_tags,tag_items,user_items=initStat(records,train_data)

user_tags, tag_items, user_items初始化完成.
user_tags大小 1860, tag_items大小 36884, user_items大小 1860


### SimpleTagBased

In [11]:
# 对用户user推荐Top-N
def recommend(user, N):
    recommend_items=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    tagged_items = user_items[user]     
    for tag, wut in user_tags[user].items():
        #print(self.user_tags[user].items())
        for item, wti in tag_items[tag].items():
            if item in tagged_items:
                continue
            #print('wut = %s, wti = %s' %(wut, wti))
            if item not in recommend_items:
                recommend_items[item] = wut * wti
            else:
                recommend_items[item] = recommend_items[item] + wut * wti
    return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

### NormTagBased

In [274]:
cc=0
def get_score(user,item):
    global cc
    user_tags_count=len(user_tags[user].keys())
    score=0
    for tag in user_tags[user].keys():
        tag_item_count=len(tag_items[tag].keys())
        if tag_items.get(tag).get(item)!=None:
        # 计算分数 累加某用户使用某标签的次数/该用户使用的全部标签的数量*该标签下某商品件数/该标签下全部商品件数
            cc=cc+1
            score=score+user_tags.get(user).get(tag)/user_tags_count*tag_items.get(tag).get(item)/tag_item_count
    return score

# 全部商品列表
all_items=set()
for user,items in train_data.items():
    all_items.update(set(items.keys()))
    
def get_recommend_items(user,N):
    t=0
    recommend_items={}
    for item in (all_items-set(user_items.get(user).keys())):
        recommend_items[item]=get_score(user,item)
        t=t+1
    print("t:",t)
    return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

In [275]:
4878672/18217

267.8087500686172

In [276]:
cc

0

In [277]:
# 这么慢？？？？
%time get_recommend_items(8,10)

t: 59496
Wall time: 1.91 s


[(15, 0.007202754900925633),
 (23702, 0.006280946268109297),
 (66188, 0.00627448884327588),
 (23377, 0.006105557776889243),
 (28750, 0.006103763987792471),
 (14937, 0.006097560975609756),
 (7835, 0.0060975609756097554),
 (21257, 0.004153575781728231),
 (27, 0.003081611441116954),
 (1855, 0.003048780487804878)]

In [285]:
18217-17981

236

In [284]:
%time recommend(8,10)

17981
Wall time: 19 ms


[(15, 0.007202754900925633),
 (23702, 0.006280946268109297),
 (66188, 0.00627448884327588),
 (23377, 0.006105557776889243),
 (28750, 0.006103763987792471),
 (14937, 0.006097560975609756),
 (7835, 0.0060975609756097554),
 (21257, 0.004153575781728231),
 (27, 0.003081611441116954),
 (1855, 0.003048780487804878)]

In [283]:
# 对用户user推荐Top-N

def recommend(user, N):
    c=0
    recommend_items={}
    user_tag_count=len(user_tags[user].keys())
    tagged_items = user_items[user]     
    for tag, wut in user_tags[user].items():
        tag_item_count=len(tag_items[tag].keys())
        for item, wti in tag_items[tag].items():
            if item in tagged_items:
#                 c=c+1/
                continue
            if item not in recommend_items:
                recommend_items[item] = wut/user_tag_count * wti/tag_item_count
                c=c+1
            else:
                recommend_items[item] = recommend_items[item] + wut/user_tag_count * wti/tag_item_count
                c=c+1
    print(c)
    return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]

In [286]:
# 使用测试集，计算准确率和召回率
def precisionAndRecall(N):
    hit = 0
    h_recall = 0
    h_precision = 0
    for user,items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        rank = recommend(user, N)
        for item,rui in rank:
            if item in items:
                hit = hit + 1
        h_recall = h_recall + len(items)
        h_precision = h_precision + N
    #print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
    # 返回准确率 和 召回率
    return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))

In [31]:
# 使用测试集，对推荐结果进行评估
def testRecommend():
    print("推荐结果评估")
    print("%3s %10s %10s" % ('N',"精确率",'召回率'))
    for n in [5,10,20,40,60,80,100]:
        precision,recall = precisionAndRecall(n)
        print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))

In [14]:
testRecommend()

推荐结果评估
  N        精确率        召回率
  5      0.829%      0.355%
 10      0.633%      0.542%
 20      0.512%      0.877%
 40      0.381%      1.304%
 60      0.318%      1.635%
 80      0.276%      1.893%
100      0.248%      2.124%


In [207]:
testRecommend()

推荐结果评估
  N        精确率        召回率


KeyboardInterrupt: 